In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [26]:
fn=pd.read_csv("Fraud_check.csv")
fn

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [27]:
fn.rename(columns={"Undergrad":"Undergrad","Marital.Status":"maratial","Taxable.Income":"income","City.Population":"population","Work.Experience":"experience","Urban":"urban"},inplace=True)

In [28]:
fn

,Undergrad,maratial,income,population,experience,urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [29]:
fn.loc[fn["income"]<=3000,"income"]="risky"
fn.loc[fn["income"]>3000,"income"]="good"

In [30]:
fn

,Undergrad,maratial,income,population,experience,urban
0,NO,Single,good,50047,10,YES
1,YES,Divorced,good,134075,18,YES
2,NO,Married,good,160205,30,YES
3,YES,Single,good,193264,15,YES
4,NO,Married,good,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,good,39492,7,YES
596,YES,Divorced,good,55369,2,YES
597,NO,Divorced,good,154058,0,YES
598,YES,Married,good,180083,17,NO


In [31]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
for column_name in fn.columns:
    if fn[column_name].dtype == object:
        fn[column_name] = le.fit_transform(fn[column_name])
    else:
        pass
fn

,Undergrad,maratial,income,population,experience,urban
0,0,2,0,50047,10,1
1,1,0,0,134075,18,1
2,0,1,0,160205,30,1
3,1,2,0,193264,15,1
4,0,1,0,27533,28,0
...,...,...,...,...,...,...
595,1,0,0,39492,7,1
596,1,0,0,55369,2,1
597,0,0,0,154058,0,1
598,1,1,0,180083,17,0


In [32]:
X=fn.iloc[:,:-1]
X

,Undergrad,maratial,income,population,experience
0,0,2,0,50047,10
1,1,0,0,134075,18
2,0,1,0,160205,30
3,1,2,0,193264,15
4,0,1,0,27533,28
...,...,...,...,...,...
595,1,0,0,39492,7
596,1,0,0,55369,2
597,0,0,0,154058,0
598,1,1,0,180083,17


In [33]:
Y=fn.iloc[:,-1]
Y

0      1
1      1
2      1
3      1
4      0
      ..
595    1
596    1
597    1
598    0
599    0
Name: urban, Length: 600, dtype: int32

In [34]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier


In [61]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(X,Y, test_size=0.2,random_state=40) 

In [62]:
num_trees = 100
max_features = 3
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(x_train,y_train) 
 

RandomForestClassifier(max_features=3)

In [63]:
preds = model.predict(x_test) # predicting on test data set 
pd.Series(preds).value_counts()

0    62
1    58
dtype: int64

In [64]:
np.mean(preds==y_test)

0.6166666666666667

In [65]:
preddata=model.predict(X)
fnew=fn
fnew['preddata']=preddata
fnew

,Undergrad,maratial,income,population,experience,urban,preddata
0,0,2,0,50047,10,1,0
1,1,0,0,134075,18,1,1
2,0,1,0,160205,30,1,1
3,1,2,0,193264,15,1,1
4,0,1,0,27533,28,0,0
...,...,...,...,...,...,...,...
595,1,0,0,39492,7,1,1
596,1,0,0,55369,2,1,1
597,0,0,0,154058,0,1,1
598,1,1,0,180083,17,0,0


In [66]:
pd.crosstab(fn["preddata"],fn["urban"])

urban,0,1
preddata,,
0,277,25
1,21,277
